In [1]:
from constants import get_top_n_market_cap_stocks
list_stocks=get_top_n_market_cap_stocks(20)

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

from data_downloader import download_and_save_stock_data
from constants import N_LARGEST_COMPANIES
from models.prophet_model import ProphetModel
from portfolio_utils import PortfolioOptimizer
from plot_utils import plot_efficient_frontier, plot_forecast, plot_monte_carlo_simulation, plot_donut_repartition


ImportError: cannot import name 'STOCKS_DIR' from 'constants' (/home/dimi/BeCode/portfolio-optimization/constants.py)

In [ ]:
stock_symbols = get_n_largest_companies(N_LARGEST_COMPANIES)
download_stock_data(stock_symbols)


In [ ]:
stock_data = {}
for stock in stock_symbols:
    stock_data[stock] = pd.read_csv(os.path.join(STOCKS_DIR, stock, 'historical_data.csv'), index_col=0, parse_dates=True)['Adj Close']

stock_returns = pd.DataFrame(stock_data).pct_change().dropna()


In [ ]:
model = ProphetModel()
model.fit(stock_returns)


In [ ]:
portfolio_optimizer = PortfolioOptimizer(stock_returns)
portfolio_optimizer.calculate_metrics()
portfolio_optimizer.optimize()


In [ ]:
plot_efficient_frontier(portfolio_optimizer.portfolio)
plot_forecast(stock_returns, model, periods=365)
plot_monte_carlo_simulation(portfolio_optimizer.portfolio, periods=365, simulations=1000)
plot_donut_repartition(portfolio_optimizer.portfolio)
